In [166]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset 

import tqdm
from torch.autograd import Variable
import argparse
import math
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

In [167]:
df = pd.read_csv('./total.csv')
df.head()

,Date,S&P,currency,gold,kospi
0,2003-12-01,1070.119995,1198.400024,402.700012,807.390015
1,2003-12-02,1066.619995,1195.000000,403.700012,807.780029
2,2003-12-03,1064.729980,1192.500000,403.899994,808.340027
3,2003-12-04,1069.719971,1190.099976,403.299988,805.130005
4,2003-12-05,1061.500000,1184.699951,406.399994,789.409973


In [168]:
# Gaussian transformation

mean_k = df['kospi'].mean()
var_k = df['kospi'].var()

df_k = df['kospi'].copy()
df_k = df_k.apply(lambda x: (x - mean_k)/var_k)

# train-test split
train_data = df_k.iloc[:int(len(df_k) * 0.8)]
test_data = df_k.iloc[int(len(df_k) * 0.8):]

train_data.shape, test_data.shape

((4256,), (1065,))

In [169]:
class Stockdataset(Dataset):
    def __init__(self, data, input_size=128, day_interval=2):
        self.data = data
        self.len = len(data)
        start_pos = (self.len - input_size -1) % day_interval
        print(self.len, start_pos, (self.len - input_size -1) // day_interval)
        X = []
        Y = []
        for i in range((self.len - input_size -1) // day_interval + 1):  # check iteration number 
            start = start_pos + i * day_interval
            end = start + input_size    # -1 due to slicing loc not iloc
            x_num = df_k.iloc[start:end].to_numpy()
            X.append(x_num)
            Y.append(df_k.iloc[end + 1])
        self.x = X
        self.y = Y
        self.len = len(X)
        # print(start, end)

    def __len__(self):
        return self.len

    def __getitem__(self, index):
        return self.x[index], self.y[index]

In [170]:
IN_DIM = 128
DAY_INT = 2
BATCH_SIZE = 64

In [171]:
train_dataset = Stockdataset(train_data, input_size=IN_DIM, day_interval=DAY_INT)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

test_dataset = Stockdataset(test_data, input_size=IN_DIM, day_interval=DAY_INT)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)
print(next(iter(test_dataset)))

4256 1 2063
1065 0 468
(array([-0.00388298, -0.00388164, -0.00387972, -0.00389074, -0.00394472,
       -0.00396055, -0.00395179, -0.00392676, -0.00393881, -0.00388748,
       -0.00383227, -0.00387131, -0.00390805, -0.0038826 , -0.0038699 ,
       -0.00389277, -0.0039019 , -0.00393394, -0.00393394, -0.00394664,
       -0.00393431, -0.00387158, -0.00387158, -0.00387158, -0.00383536,
       -0.00382561, -0.00382791, -0.00381541, -0.00382543, -0.00375292,
       -0.00373396, -0.00374207, -0.00373798, -0.00375158, -0.00374372,
       -0.00371333, -0.00369764, -0.00369764, -0.00369764, -0.00369764,
       -0.0036713 , -0.00369194, -0.00370375, -0.00372476, -0.00374183,
       -0.00371989, -0.00377146, -0.00378646, -0.00376785, -0.00373589,
       -0.00368596, -0.00367899, -0.00364624, -0.00364071, -0.00362618,
       -0.00362927, -0.00361719, -0.00364363, -0.003628  , -0.00364229,
       -0.00364218, -0.00368658, -0.00367875, -0.00368565, -0.00362193,
       -0.00362193, -0.00356771, -0.0035

In [172]:
len(test_dataset[0][0])

128

In [173]:
class TFModel(nn.Module):
    def __init__(self, iw, ow, d_model, nhead, nlayers, dropout=0.2):
        super(TFModel, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=nlayers) 
        self.pos_encoder = PositionalEncoding(d_model, dropout)

        self.encoder = nn.Sequential(
            nn.Linear(iw, d_model//2),
            nn.ReLU(),
            nn.Linear(d_model//2, d_model)
        )
        
        self.linear =  nn.Sequential(
            nn.Linear(d_model, d_model//2),
            nn.ReLU(),
            nn.Linear(d_model//2, iw)
        )

        self.linear2 = nn.Sequential(
            nn.Linear(iw, (iw+ow)//2),
            nn.ReLU(),
            nn.Linear((iw+ow)//2, ow)
        ) 

    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def forward(self, src):
        src = self.encoder(src)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src)
        output = self.linear(output)
        output = self.linear2(output)
        return output

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

def gen_attention_mask(x):
    mask = torch.eq(x, 0)
    return mask

In [174]:
device = torch.device("cuda")
lr = 1e-4
model = TFModel(IN_DIM, 1, 256, 8, 4, 0.1).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

c:\Users\alan.kwak\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [175]:
epoch = 1000
model.train()
progress = tqdm.tqdm(range(epoch))
for i in progress:
    batchloss = 0.0
    for (inputs, outputs) in train_dataloader:
        optimizer.zero_grad()
        # src_mask = model.generate_square_subsequent_mask(inputs.shape[1]).to(device)
        result = model(inputs.float().to(device))
        loss = criterion(result, outputs.float().to(device))
        loss.backward()
        optimizer.step()
        batchloss += loss
    progress.set_description("loss: {:0.6f}".format(batchloss.cpu().item() / len(train_dataloader)))

  0%|          | 0/1000 [00:00<?, ?it/s]c:\Users\alan.kwak\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\alan.kwak\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
loss: 0.000002: 100%|██████████| 1000/1000 [21:56<00:00,  1.32s/it]


In [176]:
def evaluate():
    # result = []
    # for (inputs, outputs) in test_dataloader:
    #     model.eval()
    #     inputs = inputs.float().to(device)
    #     # src_mask = model.generate_square_subsequent_mask(input.shape[1]).to(device)
    #     predictions = model(inputs)
    #     result = predictions.detach().cpu().numpy()
    #     break
    # return result
    input = next(iter(test_dataloader))[0].float().to(device)
    pred = model(input)
    return pred.detach().cpu().numpy()

In [177]:
result = evaluate()

# result = result.shape(-1, 64)
# result = result[:-1] * var_k + mean_k
# real = test_data.to_numpy() * var_k + mean_k


# plt.figure(figsize=(20,5))
# plt.plot(range(400,744),real[400:], label="real")
# plt.plot(range(744-24*7,744),result, label="predict")
# plt.legend()
# plt.show()
print(result)
len(result)

[[[-0.00058189]
  [-0.00068307]
  [-0.00070772]
  ...
  [-0.00068707]
  [-0.00063811]
  [-0.00064987]]

 [[-0.00072058]
  [-0.00068352]
  [-0.00064973]
  ...
  [-0.00065833]
  [-0.0007312 ]
  [-0.00062883]]

 [[-0.00063354]
  [-0.00059854]
  [-0.00047506]
  ...
  [-0.00058198]
  [-0.00066437]
  [-0.0006587 ]]

 ...

 [[-0.00059712]
  [-0.00060043]
  [-0.0006438 ]
  ...
  [-0.00072129]
  [-0.00054869]
  [-0.00069826]]

 [[-0.00069606]
  [-0.00061799]
  [-0.00065076]
  ...
  [-0.00064815]
  [-0.00068636]
  [-0.00064157]]

 [[-0.00063594]
  [-0.00065537]
  [-0.00063639]
  ...
  [-0.0006718 ]
  [-0.00062346]
  [-0.00071334]]]


64